In [1]:
!pip install -q -U bitsandbytes

!pip3 install -q -U peft

!pip3 install -q -U trl

!pip3 install -q -U accelerate

!pip3 install -q -U datasets

!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import bitsandbytes as bnb
print(bnb.__version__)

0.47.0


In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer #used for fine tuning
from peft import LoraConfig #peft and loraconfig used for using lora technique and fine tuning
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer #BitsAndBytesConfig used for quantization

In [4]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
model_id="google/gemma-2b"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True, #quantize for 32 bit to 4 bit
    bnb_4bit_quant_type="nf4", # 4-bit NormalFloat(nf4) is technique for quantization to convert it to 4 bit
    bnb_4bit_compute_dtype=torch.float16 #the updation of the weights in fine tuning is done on float16. when we do quantization some loss will happen so to equalize it we will do fine tuning with float16
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN']
                                             )

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
text="Quote: Imagination is more,"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto, Portugal.

I have always been fascinated by the power of imagination and the way it can transform our reality.

I


In [8]:
text="Quote: Imagination is more,"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto, Portugal.

I have always been fascinated by the power of imagination and the way it can transform our reality.

I


In [111]:
os.environ["WANDB_DISABLED"]="false"

In [112]:
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type = "CAUSAL_LM",
)

In [128]:
from datasets import load_dataset

data=load_dataset("Abirate/english_quotes")
data=data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [129]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [130]:
data['train']['author']

Column(['Oscar Wilde', 'Marilyn Monroe', 'Albert Einstein', 'Frank Zappa', 'Marcus Tullius Cicero'])

In [131]:
def formatting_func(example):
  text=f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [132]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:687: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


In [133]:
trainer.train()

Step,Training Loss
1,2.557800
2,1.625700
3,2.481000
4,2.750100
5,2.300700
6,2.474200
7,2.879600
8,2.234400
9,3.177100
10,2.215100


TrainOutput(global_step=100, training_loss=2.056935045719147, metrics={'train_runtime': 135.6703, 'train_samples_per_second': 2.948, 'train_steps_per_second': 0.737, 'total_flos': 190678527025152.0, 'train_loss': 2.056935045719147})

In [134]:
text="Quote: A woman is like a tea bag;"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: A woman is like a tea bag; you can't tell how strong she is until you put her in hot water.

I'm not sure if this is a quote or not, but I've heard it before. I've heard it used in a few different ways


In [135]:
text="Quote: Outside of a dog, a book is man's"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=25)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Outside of a dog, a book is man's best friend. Inside of a dog, it's too dark to read.

-Groucho Marx

I'm


In [136]:
text="Quote: “I am enough of an artist to"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=22)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: “I am enough of an artist to draw freely upon my imagination. Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world


In [137]:
text="Quote: “Love all, trust a few ,"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)
outputs=model.generate(**inputs,max_new_tokens=10)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: “Love all, trust a few , do wrong to none.”

“I love you
